In [87]:
import sys
sys.path.append("../../")
import utilssphering
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.config import Config

In [5]:
evalzoo_combined = pd.read_parquet("../../data/evalzoo_metrics_combined.paraquet")
# Remove site subsamples and look at just replicate scores
evalzoo_combined = evalzoo_combined[
    (~evalzoo_combined["Batch"].str.contains("siteSub")) &
    (evalzoo_combined["metric_type"] == "ref")
    ]
evalzoo_combined

,Batch,Metadata_Plate,Metadata_pert_iname,Metadata_reference_or_other,sim_scaled_mean_i_mean_i,sim_scaled_mean_i_median_i,sim_scaled_median_i_mean_i,sim_scaled_median_i_median_i,sim_ranked_relrank_mean_i_mean_i,sim_ranked_relrank_mean_i_median_i,...,sim_retrieval_r_precision_i_median_i,sim_retrieval_average_precision_i_nlog10pvalue_mean_i,sim_retrieval_average_precision_i_nlog10pvalue_median_i,sim_retrieval_average_precision_i_nlog10qvalue_mean_i,Vendor,Magnification,Binning,z_plane,metric_type,Metadata_moa
0,Scope1_MolDev_10X,Plate2_PCO_6ch_4site_10XPA,A-366,pert,0.017355,0.034813,-0.152331,-0.183784,0.505952,0.531250,...,0.142857,0.333687,0.277463,0.308864,MolDev,10,1,1,ref,None
1,Scope1_MolDev_10X,Plate3_PCO_6ch_4site_10XPA_Crest,A-366,pert,0.017355,0.034813,-0.152331,-0.183784,0.505952,0.531250,...,0.142857,0.333687,0.277463,0.308864,MolDev,10,1,1,ref,None
2,Scope1_MolDev_10X,Plate2_PCO_6ch_4site_10XPA,ABT-737,pert,1.550364,1.901986,1.559669,1.654185,0.158854,0.126488,...,0.571429,2.727806,3.008908,2.293433,MolDev,10,1,1,ref,None
3,Scope1_MolDev_10X,Plate3_PCO_6ch_4site_10XPA_Crest,ABT-737,pert,1.550364,1.901986,1.559669,1.654185,0.158854,0.126488,...,0.571429,2.727806,3.008908,2.293433,MolDev,10,1,1,ref,None
4,Scope1_MolDev_10X,Plate2_PCO_6ch_4site_10XPA,AMG-925,pert,2.544456,2.651249,2.358491,2.587136,0.101935,0.093750,...,0.714286,3.290924,4.000043,2.591954,MolDev,10,1,1,ref,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7311,Scope1_Yokogawa_US_40X_BRO0117059,BRO0117059_40x,selumetinib,pert,0.949837,1.015484,1.127078,1.253564,0.295139,0.250000,...,0.333333,1.123299,0.841086,0.885060,Yokogawa_US,40,1,12,ref,None
7312,Scope1_Yokogawa_US_40X_BRO0117059,BRO0117059_40x,sirolimus,pert,1.461585,1.534622,0.028560,0.072033,0.347222,0.361111,...,0.333333,1.253498,1.222400,0.971353,Yokogawa_US,40,1,12,ref,None
7313,Scope1_Yokogawa_US_40X_BRO0117059,BRO0117059_40x,skepinone-l,pert,1.897167,2.238411,1.718965,1.958182,0.145833,0.090278,...,0.666667,2.279437,2.832817,1.816556,Yokogawa_US,40,1,12,ref,None
7314,Scope1_Yokogawa_US_40X_BRO0117059,BRO0117059_40x,valrubicin,pert,2.094460,2.274195,2.062107,2.293297,0.083333,0.069444,...,0.666667,2.811313,3.301073,2.304229,Yokogawa_US,40,1,12,ref,None


In [27]:
# Report the top pert names wehen you aggregate across all vendors

evalzoo_combined.groupby("Metadata_pert_iname").mean().sort_values("sim_retrieval_average_precision_i_mean_i", ascending=False).reset_index()["Metadata_pert_iname"][0:5].tolist()

['AMG900', 'SCH-900776', 'PF-477736', 'RG7112', 'acriflavine']

In [26]:
# What are the top mAPs for each vendor?

all_top = []
for vend in evalzoo_combined["Vendor"].unique():
    subset_df = evalzoo_combined[evalzoo_combined["Vendor"] == vend]

    top = subset_df.groupby("Metadata_pert_iname").mean().sort_values("sim_retrieval_average_precision_i_mean_i", ascending=False).reset_index()["Metadata_pert_iname"][0:5].tolist()
    all_top += top

    print(f"Top 5 perturbations for {vend} are: {(', ').join(top)}")

pd.DataFrame(all_top).value_counts()

Top 5 perturbations for MolDev are: AMG900, SCH-900776, GSK-3-inhibitor-IX, RG7112, neratinib
Top 5 perturbations for Nikon are: THZ1, AZD2014, Compound2, AMG900, acriflavine
Top 5 perturbations for PE are: PF-477736, AMG900, APY0201, SCH-900776, AZD2014
Top 5 perturbations for Yokogawa_Japan are: PF-477736, acriflavine, Compound2, neratinib, ponatinib
Top 5 perturbations for Yokogawa_US are: PF-477736, THZ1, AMG900, APY0201, SCH-900776


AMG900                4
PF-477736             3
SCH-900776            3
APY0201               2
AZD2014               2
Compound2             2
THZ1                  2
acriflavine           2
neratinib             2
GSK-3-inhibitor-IX    1
RG7112                1
ponatinib             1
dtype: int64

In [90]:
# In what wells were these treatments added?

treatments = ["AMG900", "PF-477736", "SCH-900776", "DMSO"]

platemap = pd.read_csv("../../data/JUMP-MOA_compound_platemap_with_metadata.csv")

for tr in treatments:
    wells = platemap[platemap["pert_iname"] == tr]["well_position"].tolist()    
    print(f"{tr} is in wells: {', '.join(wells)}")

AMG900 is in wells: B19, F16, K07, O08
PF-477736 is in wells: C04, G03, L17, P18
SCH-900776 is in wells: C13, F04, I14, N09
DMSO is in wells: A11, B18, D17, D19, E07, E08, F07, F24, G20, G23, H10, I12, I13, J01, J21, K24, M05, M06, N06, N22, O14, O19, P02, P11


In [76]:
# For each vendor, which plate has the highest mAP for the selected treatments?

treatments = ["AMG900", "PF-477736", "SCH-900776"]
for vend in evalzoo_combined["Vendor"].unique():
    subset_df = evalzoo_combined[evalzoo_combined["Vendor"] == vend]
    for tr in treatments:
        top = subset_df[subset_df["Metadata_pert_iname"] == tr].sort_values("sim_retrieval_average_precision_i_mean_i", ascending=False).reset_index()["Metadata_Plate"][0:2].tolist()
        print(vend, tr, top)

MolDev AMG900 ['Plate3_PCO_6ch_4site_10XPA_Crestz', 'Plate3_PCO_6ch_4site_20XPA_Crestz']
MolDev PF-477736 ['Plate2_PCO_6ch_4site_10XPA', 'Plate3_PCO_6ch_4site_10XPA_Crest']
MolDev SCH-900776 ['Plate3_PCO_6ch_4site_10XPA_Crestz', 'Plate3_PCO_6ch_4site_20XPA_Crestz']
Nikon AMG900 ['BR00117061a', 'BR00117062a']
Nikon PF-477736 ['BR00117061a', 'BR00117062a']
Nikon SCH-900776 ['BR00117061a', 'BR00117062a']
PE AMG900 ['CP_Broad_Phenix_C_BIN1_1Plane_P1', 'CP_Broad_Phenix_C_BIN1_1Plane_P2']
PE PF-477736 ['CP_Broad_Phenix_C_BIN1_1Plane_P1', 'CP_Broad_Phenix_C_BIN1_1Plane_P2']
PE SCH-900776 ['CP_Broad_Phenix_C_BIN1_1Plane_P1', 'CP_Broad_Phenix_C_BIN1_1Plane_P2']
Yokogawa_Japan AMG900 ['20201021T092317', '20201020T134356']
Yokogawa_Japan PF-477736 ['20201021T092317', '20201020T134356']
Yokogawa_Japan SCH-900776 ['20201021T092317', '20201020T134356']
Yokogawa_US AMG900 ['BRO0117014_10x', 'BRO0117033_20xb']
Yokogawa_US PF-477736 ['BRO0117014_10x', 'BRO0117033_20xb']
Yokogawa_US SCH-900776 ['BRO0117


# MolDev
## DMSO
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_A11_s1

## AMG900
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_B19_s1

## PF-477736
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_C04_s1

## SCH-900776
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_C13_s1

# Nikon
## DMSO
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellA11

## AMG900
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellB19

## PF-477736
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellC04

## SCH-900776
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellC13

# PE
## DMSO
### A11 is r01c11
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/'CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1'/Images/r01c11f01

## AMG900
### B19 is r02c19
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/'CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1'/Images/r02c19f01


## PF-477736
### C04 is r03c04
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/'CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1'/Images/r03c04f01


## SCH-900776
### C13 is r03c13
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/'CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1'/Images/r03c13f01


# Yokogawa Japan
The Yokogawa CQ1 filenames contain well information that increments up to 384. This incrementation happens on a per row basis, so 001, 002, 003 is A01, A02, and A03. This was confirmed from the `MeasurementResult.ome.tif` metadata.

## DMSO
### A11 is W0011
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0011F0001

## AMG900
### B19 is 24 + 19 = W0043
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0049F0001


## PF-477736
### C04 is 2*24 + 4 = W0052
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0052F0001


## SCH-900776
### C13 is 2*24 + 13 = W0061
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0061F0001


# Yokogawa US
## DMSO
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_A11_T0001F001

## AMG900
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_B19_T0001F001

## PF-477736
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_C04_T0001F001

## SCH-900776
s3://cellpainting-gallery/cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_C13_T0001F001


In [91]:
# Download MolDev images

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_A11_s1", 
    "../../data/MolDev_DMSO_Plate2-PCO-6ch-4site-10XPA"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_B19_s1", 
    "../../data/MolDev_AMG900_Plate2-PCO-6ch-4site-10XPA"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_C04_s1", 
    "../../data/MolDev_PF-477736_Plate2-PCO-6ch-4site-10XPA"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_C13_s1", 
    "../../data/MolDev_SCH-900776_Plate2-PCO-6ch-4site-10XPA"
    )
    

Downloading: cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_A11_s1_w1.TIF
Downloading: cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_A11_s1_w2.TIF
Downloading: cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_A11_s1_w3.TIF
Downloading: cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_A11_s1_w4.TIF
Downloading: cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PCO-6ch-4site-10XPA_A11_s1_w5.TIF
Downloading: cpg0002-jump-scope/source_4/images/2020_11_06_Scope1_MolDev/images/Plate2-PCO-6ch-4site-10XPA_Plate_20092/TimePoint_1/Plate2-PC

In [92]:
# Download Nikon images

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellA11", 
    "../../data/Nikon_DMSO_BR00117061a"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellB19", 
    "../../data/Nikon_AMG900_BR00117061a"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellC04", 
    "../../data/Nikon_PF-477736_BR00117061a"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellC13", 
    "../../data/Nikon_SCH-900776_BR00117061a"
    )
    

Downloading: cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellA11_ChannelDAPI Camera,FITC Camera,TRITC Camera,Cy5 Camera_Seq0010.nd2
Downloading: cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellB19_ChannelDAPI Camera,FITC Camera,TRITC Camera,Cy5 Camera_Seq0029.nd2
Downloading: cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellC04_ChannelDAPI Camera,FITC Camera,TRITC Camera,Cy5 Camera_Seq0051.nd2
Downloading: cpg0002-jump-scope/source_4/images/2020_11_24_Scope1_Nikon/201026_kag_Painting_Test_BR00117061a__20201110_105314_621/WellC13_ChannelDAPI Camera,FITC Camera,TRITC Camera,Cy5 Camera_Seq0060.nd2


In [94]:
# Download PE images

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1/Images/r01c11f01", 
    "../../data/PE_DMSO_CP_Broad_Phenix_C_BIN1_1Plane_P1"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1/Images/r02c19f01", 
    "../../data/PE_AMG900_CP_Broad_Phenix_C_BIN1_1Plane_P1"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1/Images/r03c04f01", 
    "../../data/PE_PF-477736_CP_Broad_Phenix_C_BIN1_1Plane_P1"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1/Images/r03c13f01", 
    "../../data/PE_SCH-900776_CP_Broad_Phenix_C_BIN1_1Plane_P1"
    )
    

Downloading: cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1/Images/r01c11f01p01-ch1sk1fk1fl1.tiff
Downloading: cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1/Images/r01c11f01p01-ch2sk1fk1fl1.tiff
Downloading: cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1/Images/r01c11f01p01-ch3sk1fk1fl1.tiff
Downloading: cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1/Images/r01c11f01p01-ch4sk1fk1fl1.tiff
Downloading: cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_PE/images/C_BIN1_1Plane/CP Broad Phenix C BIN1 1Plane P1__2020-11-11T09_26_12-Measurement 1/Images/r01c11f01p01-ch5sk1fk1fl1.tiff
Downloading: cpg0002

In [97]:
# Download Yokogawa Japan images

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0011F0001", 
    "../../data/Yokogawa_Japan_DMSO_20201021T092317"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0049F0001", 
    "../../data/Yokogawa_Japan_AMG900_20201021T092317"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0052F0001", 
    "../../data/Yokogawa_Japan_PF-477736_20201021T092317"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0061F0001", 
    "../../data/Yokogawa_Japan_SCH-900776_20201021T092317"
    )
    

Downloading: cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0011F0001T0001Z000C1.tif
Downloading: cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0011F0001T0001Z000C2.tif
Downloading: cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0011F0001T0001Z000C3.tif
Downloading: cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0011F0001T0001Z000C4.tif
Downloading: cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0011F0001T0001Z000C5.tif
Downloading: cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0011F0001T0001ZSUMC1.tif
Downloading: cpg0002-jump-scope/source_4/images/2020_10_27_Scope1_YokogawaJapan/images/20201021T092317_/Projection/W0011F0001T0001ZSUMC2.tif
Downloading: 

In [98]:
# Download Yokogawa US images

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_A11_T0001F001", 
    "../../data/Yokogawa_US_DMSO_BRO0117014"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_B19_T0001F001", 
    "../../data/Yokogawa_US_AMG900_BRO0117014"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_C04_T0001F001", 
    "../../data/Yokogawa_US_PF-477736_BRO0117014"
    )

utilssphering.download_from_cpg(
    "cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_C13_T0001F001", 
    "../../data/Yokogawa_US_SCH-900776_BRO0117014"
    )
    

Downloading: cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_A11_T0001F001L01A01Z01C02.tif
Downloading: cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_A11_T0001F001L01A01Z01C03.tif
Downloading: cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_A11_T0001F001L01A02Z01C04.tif
Downloading: cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_A11_T0001F001L01A02Z01C05.tif
Downloading: cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-singleplane-10x_20201026_113115/PECCU/PECCU_A11_T0001F001L01A03Z01C01.tif
Downloading: cpg0002-jump-scope/source_4/images/2020_11_16_Scope1_YokogawaUS/images/BRO0117014-3D-2x2x1-single